[Home](../index.ipynb) / Web-Server
***
<span style="font-size:20pt;">Web-Server</span>

***
# ESP32 / ESP8266 als Accesspoint

**Achtung:** ESP8266: Ein **eigenes** Passwort (statt dem default Passwort **micropythoN**) braucht **mindestens 8 Zeichen**, andernfalls bricht das Programm ab mit der  
Fehlermeldung: _OSError: can't set AP config_.

Nach dem Start der Programme das WLAN des Controllers auswählen und die auf der Console angezeigte IP-Adresse ansurfen (meist 192.168.4.1).

In [ ]:
#%serialconnect --port=COM3 # Windows
%serialconnect # Linux or Windows with one COM port 


#######################################
# Constants

SSID        = "controller_a.fuchs"
PASSWORD    = '1231231123' # default is 'micropythoN', attension: at least 8 chars, otherwise error: 'OSError: can't set AP config'

MAX_CLIENTS = 5 # Good maximal ammount of posiible clients

ID_LED      = 25


STR_HTML_TOP = r"""<!DOCTYPE html>
<html lang="de">
<head>
    <meta charset="utf-8">
    <title>Web Server</title>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="icon" href="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAIAAACQd1PeAAAADElEQVQI12P4//8/AAX+Av7czFnnAAAAAElFTkSuQmCC">
    <style>
        html{font-family:Helvetica;display:inline-block;margin:0px auto; text-align: center;}
        p{font-size: 2rem;}
        a{display:inline-block;border:none;border-radius:20px;color:white;padding:16px 20px;margin:2px;text-decoration:none}
        .clsOff{background-color:#eebb33;}
        .clsOn{background-color:#8080ee;}
    </style>
</head>
<body>
  <h1>Web Server</h1>
  <p>Switch LED:</p>"""
  

STR_HTML_BOTTOM = r"""</body>
</html>"""

#######################################
# Variables and init

from machine import Pin

led = Pin( ID_LED, Pin.OUT )


#======================================
# Typical code for setting up the controller as access point.
# At the end we get a socket listening for clients to connect:

import network
import socket


wlanAccessPoint = network.WLAN( network.AP_IF ) # create access-point interface
wlanAccessPoint.config( essid = SSID, password = PASSWORD, max_clients = MAX_CLIENTS )
wlanAccessPoint.active( True )                  # activate the interface

print( wlanAccessPoint.ifconfig()[0] )

objSocket = socket.socket( socket.AF_INET, socket.SOCK_STREAM )

try:
    objSocket.bind( ( '', 80 ) )
    objSocket.listen( MAX_CLIENTS )
except Exception as _e:
    print( _e )


#######################################
# Loop
# Listen for connecting clients and
# handle requests

try:
    while True:
        try :
            socketFrom, addressFrom = objSocket.accept() # The return value is a pair (socketFrom, addressFrom) where socketFrom is a new socket object usable to send 

            strRequest = str( socketFrom.recv( 1024 ) )

            iRequestPathStart = strRequest.find( "GET " ) # Every http request starts with "GET "

            if iRequestPathStart >= 0 :                    # followed by the request-path delimited with a ' ' charakter.
                iRequestPathStart += 4

                iRequestPathEnd = strRequest.find( " ", iRequestPathStart )

                if iRequestPathEnd >= 0 :
                    strPath = strRequest[ iRequestPathStart : iRequestPathEnd ]
                    bIsOn   = False
                    # Now we have a valid request-path:
                    # print( strPath )
                    
                    if strPath == "/led/on" :
                        strPath = "/led/off"
                        led.on()
                        bIsOn = True
                    else :
                        strPath = "/led/on"
                        led.off()
                    
                    # This header is allways needed:
                    socketFrom.send   ( "HTTP/1.1 200 OK\n" )
                    socketFrom.send   ( "Content-Type: text/html; charset=UTF-8\n" )
                    socketFrom.send   ( "Connection: close\n\n" )
                    
                    # Now send html content:
                    socketFrom.sendall( STR_HTML_TOP   )
                    socketFrom.sendall( "<p><a class='cls{1}' href='{0}'>{1}</a></p>".format( strPath, "Off" if bIsOn else "On" ) )
                    socketFrom.sendall( STR_HTML_BOTTOM )

        except Exception as _e:
            print( _e )

        finally :
            try   : socketFrom.close()
            except: pass
        
except KeyboardInterrupt:
    pass

In [ ]:
#%serialconnect --port=COM3 # Windows
%serialconnect # Linux or Windows with one COM port 


#######################################
# Constants

SSID        = "controller_a.fuchs"
PASSWORD    = '1231231123' # default is 'micropythoN', attension: at least 8 chars, otherwise error: 'OSError: can't set AP config'

MAX_CLIENTS = 5 # Good maximal ammount of posiible clients

ID_LED      = 25



STR_HTML_TOP = r"""<!DOCTYPE html>
<html lang="de">
<head>
    <meta charset="utf-8">
    <title>Web Server</title>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="icon" href="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAIAAACQd1PeAAAADElEQVQI12P4//8/AAX+Av7czFnnAAAAAElFTkSuQmCC">
    <style>
        html{font-family:Helvetica;display:inline-block;margin:0px auto;text-align:center;}
        input{display:inline-block;border-radius:20px;padding:12px 12px;margin:2px;font-size:1.2rem;}
    </style>
</head>
<body>
    <h1>Web Server</h1>
    <form action="/text">
        <input type="text" id="idText" name="idText" value="
"""

STR_HTML_BOTTOM = r""""><br><br>
        <input type="submit" value="Display text">
    </form>  
</body>
</html>"""


#######################################
# Variables and init

import display

display = display.Display()

display.clear()


# source: <a href="https://forum.micropython.org/memberlist.php?mode=viewprofile&u=552&sid=825359ab512e29152debfb452a578c20">Spotlight kid</a>
def uudecode( str ):
    bits = str.replace( "+", " " ).split('%')
    if len(bits) == 1:
        return str
    res = bytearray(bits[0])
    append = res.append
    extend = res.extend

    for item in bits[1:]:
        try:
            append(int(item[:2], 16))
            extend(item[2:])
        except KeyError:
            append(b'%')
            extend(item)

    return res.decode('utf-8')


#======================================
# Typical code for setting up the controller as access point.
# At the end we get a socket listening for clients to connect:

import network
import socket


wlanAccessPoint = network.WLAN( network.AP_IF ) # create access-point interface
wlanAccessPoint.config( essid = SSID, password = PASSWORD, max_clients = MAX_CLIENTS )
wlanAccessPoint.active( True )                  # activate the interface

print( wlanAccessPoint.ifconfig()[0] )

objSocket = socket.socket( socket.AF_INET, socket.SOCK_STREAM )

try:
    objSocket.bind( ( '', 80 ) )
    objSocket.listen( MAX_CLIENTS )
except Exception as _e:
    print( _e )


#######################################
# Loop
# Listen for connecting clients and
# handle requests

try:
    while True:
        try :
            socketFrom, addressFrom = objSocket.accept() # The return value is a pair (socketFrom, addressFrom) where socketFrom is a new socket object usable to send 

            strRequest = str( socketFrom.recv( 1024 ) )

            iRequestPathStart = strRequest.find( "GET " ) # Every http request starts with "GET "

            if iRequestPathStart >= 0 :                    # followed by the request-path delimited with a ' ' charakter.
                iRequestPathStart += 4

                iRequestPathEnd = strRequest.find( " ", iRequestPathStart )

                if iRequestPathEnd >= 0 :
                    strText = strRequest[ iRequestPathStart + len("/text?idText="): iRequestPathEnd ]
                    #print( strText )
                    strText = uudecode( strText )
                    #print( strText )
                    
                    display.scroll(0,-12)
                    display.fill_rect(0,52,127,63,0)
                    display.text( strText.replace("ä","ae").replace("ö","oe").replace("ü","ue").replace("Ä","Ae").replace("Ö","Oe").replace("Ü","Ue").replace("ß","ss"), 0, 52 )
                    display.show()
                    
                    # This header is allways needed:
                    socketFrom.send   ( "HTTP/1.1 200 OK\n" )
                    socketFrom.send   ( "Content-Type: text/html; charset=UTF-8\n" )
                    socketFrom.send   ( "Connection: close\n\n" )
                    
                    # Now send html content:
                    socketFrom.sendall( STR_HTML_TOP )
                    socketFrom.send   ( strText )
                    socketFrom.sendall( STR_HTML_BOTTOM )

        except Exception as _e:
            print( _e )

        finally :
            try   : socketFrom.close()
            except: pass
        
except KeyboardInterrupt:
    pass